In [1]:
import os
import pickle

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [1]:
from modules.logging.format_utils import format_measures
from modules.logging.logger import DefaultLogger
from modules.collecting.results_collector import DataFrameCollector
from modules.algorithms.nn.OSLPP_NN_UTILS import train_osda
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode

results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': [], 'tops': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
#                 for weights in [[0.33, 0.33, 0.33], [0.2, 0.4, 0.4], [0.4, 0.2, 0.4], [0.4, 0.4, 0.2]]:
                 for tops in [[SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY],
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.MARGIN],
                                [SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN],
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN]]:
                    for seed in range(5):
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                    dataset='DomainNet_DCC', source=source, target=target,
                                    num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                        metrics = train_osda(params, lr, epochs, SelectRejectMode.TOPS, seed, common, tgt_private, logger)
                        results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'tops': tops, 'epochs': epochs}, ignore_index=True)
results.get_result().to_csv('./all_results/results_weighted_scores/dcc__conf__small__NEW__combined_tops__nn_raw.csv', header=True, index=False)

NameError: name 'config' is not defined